# 1、Gym介绍

Gym是OpenAI于2016年推出的用于强化学习开发和测试的开发套件，定义了一套用于强化学习的API标准，内置大量的environments。Gym的作用类似于图像识别中的ImageNet，用于不同算法的对比。
但因为OpenAI投入的资源比较少，后期缺少维护，于2021年迁移到了Gymnasium项目，[fork的版本是0.26.2](https://github.com/Farama-Foundation/Gymnasium/releases?page=1)，尽管换了项目名，Gym 0.26.2版本和Gymnasium接口是兼容的，后文的Gym代指Gymnasium。主要支持Linux和mac，Windows上不保证能运行。


# 2、Gym 安装

安装比较简单，pip或conda安装任选一个。

```
pip install Gymnasium
conda install Gymnasium
```
gym内置的部分环境（游戏）需要手动安装一些依赖，需再执行以下两条命令
```
pip install gymnasium[box2d]
pip install gymnasium[classic-control]
```

另外会用到matplotlib画训练曲线图，使用如下方式安装：
```
pip install matplotlib
```

安装之后使用以下方式查看gym版本，需要大于0.26.2。


In [ ]:
import gymnasium as gym
print(gym.__version__)

# 3、Gym使用
先来看官方文档首页给的例子：[月球着陆器LunarLander-v2](https://gymnasium.farama.org/environments/box2d/lunar_lander/)。着陆器会从窗口中心以一个随机的初始方向和速度往下坠，通过控制3个引擎控制着陆器到达指定着陆点。因采用的随机策略，着陆器基本不可能到达着陆点，碰到地面后游戏会重新开始，地形也会随机切换。

In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v3", render_mode="human")   # 指定LunarLander-v3环境，使用human可视化渲染模式
observation, info = env.reset(seed=42)                  # 环境初始化
for _ in range(500):                   # step数，使用随机策略一轮游戏大概要100步
   action = env.action_space.sample()  # 使用某种策略返回一个动作，此处使用随机策略
   observation, reward, terminated, truncated, info = env.step(action)    # 环境接收动作后进入到下一个状态，并返回奖励等信息

   if terminated or truncated:         # 如果游戏结束，重新开始
      observation, info = env.reset()

env.close()
print("LunarLander done")

上面的代码已经包含了实验用到的所有gym接口，跟强化学习任务的5个关键组成对应：agent, environment, observation space, action space, and reward function。重点是step函数，env接受agent的action进行状态的迁移并且返回奖励。gym内置环境包含了Classic Control、Box2D、Toy Text等类别的环境，不同的环境，接口都是相同的，使用方式也是相同，这也是gym库的意义，定义一套API标准，方便算法的开发和运行，不同的是每个环境的状态空间、动作空间。接口的具体说明请参考：[https://gymnasium.farama.org/api/env/](https://gymnasium.farama.org/api/env/)

# 4、实验环境：Cart Pole
Cart Pole由一根杆子通过非驱动接头连接到小车上，小车沿着无摩擦的轨道移动，摆锤垂直放置在小车上，目标是通过在小车上向左和向右施加力来平衡杆。

以下代码使用随机策略进行Cart Pole游戏，可以看到每轮10-40步左右就结束了。实验的任务就是使用强化学习算法学习到策略移动小车能够使得杆子平衡住。

In [ ]:
import gymnasium as gym
env = gym.make("CartPole-v1", render_mode="human")   
observation, info = env.reset(seed=42)  
for i in range(3):
   print("********episode %d********" % i)
   step = 0
   while True:                
        action = env.action_space.sample()  
        observation, reward, terminated, truncated, info = env.step(action) 
        if step == 0:
            print("action: %d, reward: %d" %(action, reward) )
            print("observation:", observation)
        step += 1
        if terminated or truncated:    
            observation, info = env.reset()
            print("game over after %d step\n" % step)
            break

env.close()

## 4.1 Cart Pole介绍

动作空间是(1,)形状的ndarray，即只有一个元素的一维数组，取值离散只有{0,1}两种情况，0是往左推小车，1是往右推小车。

状态空间是(4,)形状的ndarray，即有4个元素的一维数组，每个元素取值连续，代表的含义如下：

| Num | Observation           | Min     | Max   |
|-----|-----------------------|---------|-------|
| 0   | Cart Position         | -4.8    | 4.8   |
| 1   | Cart Velocity         | -Inf    | Inf   |
| 2   | Pole Angle            | ~-0.418 rad | 0.418 rad|
| 3   | Pole Angular Velocity | -Inf    | Inf   |

游戏的目标是尽可能久地保持杆子竖直向上，所以每一步的奖励是+1，包括终止步骤。对于v1版本即`CartPole-v1`，奖励的上限是500，即能连续完成500步游戏成功。

游戏结束有两种情况，一种游戏失败即Termination，杆子的角度大于$\pm 12 ^\circ$ 或者车的位置超过出了显示窗口范围即$\pm 2.4$，一种是游戏顺利结束即Truncation，回合长度大于500。

动作空间和状态空间常用的有gymnasium.spaces.Discrete和gymnasium.spaces.Box两种数据类型。
1. Discrete(n, seed=None,start=0)对应于一维离散空间，三个参数，n是元素的个数，元素都是整型，seed是伪随机数发生器PRNG的种子，start是最小的元素值，sample每次返回一个整数。
2. Box(low, high,shape=None,dtype=np.float32,seed=None)对应于多维空间，表示n个闭区间的笛卡尔积，每一个维度可以用一个最低值和最大值来约束，当然也要知道维数。low是区间的下界，high是区间的上界，shape是sample返回值的形状，没指定会自动推导，dtype是元素类型，float型对应连续空间，int型对应离散空间，seed是伪随机数种子。low和high参数为标量时，则所有维度的下/上限都相同。low和high参数为矢量时，将分别指定每个区间的下/上限。

官方说明可阅读：[Cart Pole](https://gymnasium.farama.org/environments/classic_control/cart_pole/)


In [ ]:
# Discrete示意
from gymnasium.spaces import Discrete
observation_space = Discrete(2, seed=42) # {0, 1}
print(observation_space.sample())
print(observation_space.sample())
print(observation_space.sample())
observation_space = Discrete(3, start=-1, seed=42)  # {-1, 0, 1}
print(observation_space.sample())
print(observation_space.sample())
print(observation_space.sample())

In [ ]:
# Box空间示意
from gymnasium.spaces import Box
import numpy as np

# 每个维度的界限相同，每次sample会生成(3,4)的矩阵，每个元素的范围是[-1.0, 2.0]
observation_space = Box(low=-1.0, high=2.0, shape=(3, 4), dtype=np.float32) 
print(observation_space.sample())
print(observation_space.sample())

# 每个维度的界限独立,每次sample会生成(2,)的数组，第1个元素的范围是[-1.0, 0.0]，第2个元素的范围是[2.0, 4.0]
observation_space = Box(low=np.array([-1.0, 2.0], dtype=np.float32), high=np.array([0.0, 4.0],dtype=np.float32), dtype=np.float32)  
print(observation_space.sample())
print(observation_space.sample())

# CartPole-v1 状态空间的范围跟上表中一致，Inf用3.4e+38表示
env = gym.make("CartPole-v1", render_mode="human")   
print("\nCartPole-v1space:")
print(env.observation_space)  
print(env.observation_space.sample())
print(env.action_space)

本节无编码任务，只需阅读、执行代码，熟悉gym的Cart Pole环境。

参考：
1. https://gymnasium.farama.org/
2. https://github.com/Farama-Foundation/Gymnasium
3. https://github.com/openai/gym
4. https://wandb.ai/mukilan/intro_to_gym/reports/A-Gentle-Introduction-to-OpenAI-Gym--VmlldzozMjg5MTA3